<a href="https://colab.research.google.com/github/freejae98/freejae98.github.io/blob/master/2022_10_18_f_0)_MI_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%88%98%EC%A7%91_22_1019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#이 부분은 처음 한번만 실행하면 됌.
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [7]:

# -*- coding: UTF-8 -*-
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
#Colab에선 웹브라우저 창이 뜨지 않으므로 별도 설정한다.
 
# options = webdriver.ChromeOptions()
# options.add_argument('--headless')        # Head-less 설정
# options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')
# browser = webdriver.Chrome('chromedriver', options=options)


# 추가 라이브러리

import requests
from bs4 import BeautifulSoup
import csv
import re


from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

import pandas as pd
# from datetime import datetime
import datetime

import requests
import json
from pandas import json_normalize

import numpy as np


from typing import Any
from urllib.parse import urljoin
from datetime import date, timedelta
from pprint import pprint

from openpyxl import load_workbook

# warning msg 무시
import warnings
warnings.filterwarnings('ignore')
 

In [8]:


# S1 모듈 - Platts API 모듈 ---------------------------------------------------------------------------------------------------------------------
def S1_data(Input_Date, num_days, company):
    
    # platts_test_22-1006_수정.py 에서 가져온 모듈
    
    print("[S1 - Platts API]")
    
    
    # DATE_FORMAT = '%Y%m%d'
    NETLOC = 'https://api.platts.com'

    PROXIES = {'http': "http://90.8.50.30:8080/", 'https': "http://90.8.50.30:8080/"}
    
    
    
    APPKEY = "EyldIGwzyApJUvnBgLCK"
    FIELDS = 'close,unspecified'

    FILEPATH = "./market_data.ods"


    # def insert_date_column(symbols: dict[str, dict[str, list[dict[str, Any]]]], read_date: date) -> list[dict[str, Any]]:
    #     return [{'date': read_date.isoformat()} | symbol for symbol in symbols["Symbols"]["Symbol"]]


    def market_data_v1(company, date_filter, symbols, filepath, 
                       fields = FIELDS, appkey = APPKEY, proxies = PROXIES):
        headers = {
            'accept': 'application/json',
            'appkey': appkey,
        }

        filter = f'Symbol[]={symbols}^{date_filter}'
        
        params = {
            'Fields': fields,
            'Filter': filter,
            'PageSize': 10000
        }
        
        
        # try:
        if company == True:
            response = requests.get(urljoin(NETLOC, '/marketdata/v3/symbolData'),
                                    headers=headers, params=params, 
                                    proxies=proxies, verify=False) 
        else:
            response = requests.get(urljoin(NETLOC, '/marketdata/v3/symbolData'),
                                    headers=headers, params=params)
        # except:
        #     pass
            # response = requests.get(urljoin(NETLOC, '/marketdata/v3/symbolData'),
            #                         headers=headers, params=params)
        
        
        return response.json()["Content"]["MarketData"]["SymbolData"]["Rows"]["Row"]
        


    # 받아올 데이터 - Symbols
    SYMBOLS = "PLVHA00,SB01032,SB01063,SB01083,SB01084,SB01119,SB01125,SB01126,SB01142,SB01152,SB01180,SB01195,SB01233,SB01261,SBO1001,STBLB00,STCBZ02,STHAM00,STHGM00,STHRE00,STHRZ02,STHSA00,STPGM00,STRAM00,STRGM00,TS01011,TS01043,TS01046"


    input_date = Input_Date    # "2022-9-21"  # 기준일자 

    dinput_date = datetime.datetime.strptime(input_date, '%Y-%m-%d')  # datetime 변경

    # input_date 기준 
    datelist = pd.date_range(dinput_date+datetime.timedelta(days=-num_days+1), dinput_date)  #최근 5일에 대해 확인

    # date를 처리 위한 string 변환
    strlist = list(map(str, datelist))  # date -> string 변환



    # (Start) Platts API 값을 받아오는 부분 -----------------------------------------------------------------------------------
    pd_temp_i = pd.DataFrame()


    for one_date in strlist:   # 휴일 날짜에 못가져오는 것을 거를 수 있게 날짜를 하나씩 나눠 처리 (try 연동)

        s_date = str(one_date[:10]).split("-")
        s_year, s_mon, s_day = (s_date[0], s_date[1].zfill(2), s_date[2].zfill(2))
        
        ss_date = s_year + s_mon + s_day
        
        # date_filter = "Date>=20220922^Date<=20220922"
        date_filter = "Date>="+ss_date+"^Date<="+ss_date
        
        result = market_data_v1(company, date_filter, SYMBOLS, FILEPATH, FIELDS, APPKEY, PROXIES)

        
        try:
            rows = result[0]['Symbols']['Symbol']
            # print(rows)
            # print(str(i) + " - ", str(len(rows)))
            
            pd_temp_j = pd.DataFrame()
            
            
            for j in range(0, len(rows)):
                
                row = rows[j]
                df = pd.DataFrame(row, index= [0])   #"2022-10-4"])
                # print(j, " : ", df)
                
                if df.iloc[0, 0] == "PLVHA00":
                    temp = df.iloc[[0], [0,3]]
                else:
                    temp = df.iloc[[0], [0,2]]
                    
                    
                t_temp = pd.DataFrame(index=range(1,2), columns={temp.iloc[0,0]})
                t_temp.iloc[0,0] = temp.iloc[0,1]
                t_temp['Date'] = ss_date  #datelist[i]
                
                t_temp = t_temp.astype({'Date':'datetime64'})        
                t_temp.set_index('Date', inplace=True)
                
                pd_temp_j = pd.concat([pd_temp_j, t_temp], axis = 1)
                
            pd_temp_i = pd.concat([pd_temp_i, pd_temp_j], axis = 0)
            
        except:
            pass

    # (End) Platts API 값을 받아오는 부분 -----------------------------------------------------------------------------------            
            
    # print(pd_temp_i)

    # pd_temp_i.info()
    # pd_temp_i["PLVHA00"].unique()
    sd1 = pd_temp_i.replace(['#N/A','# Not Applicable'], '0')


    nsd1 = sd1.astype('float')     # 최종 결과물
    
    print(nsd1)
    print()
    
    # write_file(f_writer, 1, nsd1)
    
    return nsd1

    
    
    


# S2 모듈 - 해운지수 ---------------------------------------------------------------------------------------------------------------------
def S2_data():
    print("[S2 - 해운지수]")
    
    browser.get("http://www.shippingnewsnet.com/sdata/page.html")       #크롬 드라이버에 url 주소 넣고 실행
    time.sleep(3)
    # browser.execute_script("window.open('http://www.shippingnewsnet.com/sdata/page.html', '_blank');")       #크롬 드라이버에 url 주소 넣고 실행
    soup = BeautifulSoup(browser.page_source, "lxml")
    # browser.quit()
    
    tbody_lst = soup.find("table", attrs={"id":"graph"}).find_all("tbody", limit=5) #.find_all("td")  #.find("tr").find_all("td")
    # print(td_lst)
    
    
    # 데이터 변환 시작
    pd_temp = pd.DataFrame()
    cols = ['Date', 'BDI', 'BCI', 'BPI', 'BSI']
    
    
    for index, tbody in enumerate(tbody_lst):
        td_lst = tbody.find_all("td")
        row = [td.get_text().strip().replace(',', '') for td in td_lst]
        # print(index," : ", row)
                
        ps_row = pd.Series(row, index=cols)
        pd_temp = pd.concat([pd_temp, ps_row], axis = 1)
        
        
    sd2 = pd_temp.T

    sd2 = sd2.astype({'Date':'datetime64'})
    sd2.set_index('Date', inplace=True)
        
    nsd2 = sd2.astype('float')
    # nsd2.info()
    

# =============================================================================
# 0  :  ['2022-09-22', '1,720', '1,929', '1,998', '1,637']
# 1  :  ['2022-09-21', '1,746', '2,021', '2,021', '1,605']
# 2  :  ['2022-09-20', '1,729', '1,994', '2,023', '1,580']
# 3  :  ['2022-09-16', '1,553', '1,519', '1,990', '1,551']
# 4  :  ['2022-09-15', '1,612', '1,637', '2,090', '1,529']
# 5  :  ['2022-09-14', '1,595', '1,565', '2,145', '1,505']

# =============================================================================
#------------------------------------------------------------------------------------------------------------------------------

    print(nsd2)
    print()
    
    # write_file(f_writer, 2, nsd2)
    
    
    return nsd2
    
# nsd2 출력 =============================================================================
#              BDI   BCI   BPI   BSI
# Date                              
# 2022-09-23  1816  2206  1995  1652
# 2022-09-22  1720  1929  1998  1637
# 2022-09-21  1746  2021  2021  1605
# 2022-09-20  1729  1994  2023  1580
# 2022-09-16  1553  1519  1990  1551
# =============================================================================



# S3 모듈 - 원유가 ---------------------------------------------------------------------------------------------------------------------
def S3_data():
    print("[S3 - 원유가]")

    browser.get("https://www.opinet.co.kr/glopcoilSelect.do")       #크롬 드라이버에 url 주소 넣고 실행
    # browser.execute_script("window.open('https://www.opinet.co.kr/glopcoilSelect.do', '_blank');")
    # time.sleep(3)
    
    soup = BeautifulSoup(browser.page_source, "lxml")
    # browser.quit()
    
    tbody_lst = soup.find("tbody", attrs={"id":"tbody2"}).find_all("tr") #.find_all("td")  #.find("tr").find_all("td")
    # print(td_lst)
    
    
    # 데이터 변환 시작
    pd_temp = pd.DataFrame()
    cols = ['Date', 'Dubai', 'Brent', 'WTI']
    
    
    for index, tbody in enumerate(tbody_lst):
        td_lst = tbody.find_all("td")
        row = [td.get_text().strip() for td in td_lst]
        
        ps_row = pd.Series(row, index=cols)
        pd_temp = pd.concat([pd_temp, ps_row], axis = 1)
        
        # print(index," : ", data)
    
    sd3 = pd_temp.T
    
    sd3['Date'] = sd3['Date'].apply(lambda _ : datetime.datetime.strptime(_, '%y년%m월%d일'))
    

    sd3 = sd3.astype({'Date':'datetime64'})
    sd3.set_index('Date', inplace=True)
        
    nsd3 = sd3.astype('float')
    # nsd3.info()
    
    
# =============================================================================
# 0  :  ['22년09월21일', '92.77', '89.83', '82.94']
# 1  :  ['22년09월22일', '91.21', '90.46', '83.49']
# =============================================================================

#------------------------------------------------------------------------------------------------------------------------------

    print(nsd3)
    print()
    
    # write_file(f_writer, 3, nsd3)
    
    return nsd3
    
# nsd3 출력 =============================================================================
#             Dubai  Brent   WTI
# Date                          
# 2022-09-22  91.21  90.46 83.49
# 2022-09-23  88.82  86.15 78.74
# =============================================================================
    


# S4 모듈 - 환율 ---------------------------------------------------------------------------------------------------------------------
def S4_data(Input_Date, max_try):   # 날짜를 입력받게 해야 함
    print("[S4 - 환율]")

    Data_chk = 0     # loop 용 (while) : 0 처음, 1 이전일 체크, 2 완료
    i_count = 0      # loop max count 목적
    
    while (Data_chk < 2 and i_count < max_try):
        
        if Data_chk == 0:
            #처음 날짜 입력 받음
            input_date = Input_Date  #"2022-10-4"            
            
        else:   # Data_chk == 1 일때
            input_date = datetime.datetime.strptime(input_date, '%Y-%m-%d')

            input_date = input_date+datetime.timedelta(days=-1)
            
            input_date = input_date.strftime("%Y-%m-%d")   # 전날을 string 변환

        
        t_date = input_date.split("-")
        t_year, t_mon, t_day = (t_date[0], t_date[1].lstrip("0"), t_date[2])
        
        browser.get("https://spot.wooribank.com/pot/Dream?withyou=FXXRT0011")       #크롬 드라이버에 url 주소 넣고 실행
        # browser.execute_script("window.open('https://spot.wooribank.com/pot/Dream?withyou=FXXRT0011', '_blank');")       #크롬 드라이버에 url 주소 넣고 실행
        # time.sleep(2)
        # time.sleep(5)
        
        
        
        # 날짜 선택하는 부분이 여기 들어와야 할 것 - year는 일단 제외 (포함 필요)
        browser.find_element(By.XPATH, f'/html/body/div[2]/div[2]/div[3]/form[1]/fieldset/table/tbody/tr[2]/td/div/select[1]/option[{int(t_year[2:])+1}]').click()   # 월 
        browser.find_element(By.XPATH, f'/html/body/div[2]/div[2]/div[3]/form[1]/fieldset/table/tbody/tr[2]/td/div/select[2]/option[{t_mon}]').click()   # 월 
        browser.find_element(By.XPATH, f'/html/body/div[2]/div[2]/div[3]/form[1]/fieldset/table/tbody/tr[2]/td/div/select[3]/option[{t_day}]').click()   # 월 
        
        
        browser.find_element(By.XPATH, '//*[@id="frm"]/fieldset/div/span/input').click()    # 조회버튼 클릭
        
        
        # 페이지 로딩까지 기다림
        try:
            elem = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.ID, "fxprint")))
            # 성공했을 때 동작 수행    
            # print(elem.text) # 첫번째 결과 출력
        except:
            pass
            # print("find next page")
            # browser.quit()
        
        
        soup = BeautifulSoup(browser.page_source, "lxml")
        
        # s_date = browser.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[3]/div[2]/div[1]/div/div/dl/dd[2]').text    # 조회일자
        
        
        soup.find("table", attrs={"class":"tbl-type-1 txt-c mb20 ui-set-tbl-type"}).find("tbody")
        
        tr_lst = soup.find("table", attrs={"class":"tbl-type-1 txt-c mb20 ui-set-tbl-type"}).find("tbody").find_all("tr", limit=30)  #.find("tr").find_all("td")
        # print(tr_lst)
        
        
        # 데이터 변환 시작
        pd_temp = pd.DataFrame()
        # cols = ['Date', 'BDI', 'BCI', 'BPI', 'BSI']
            
        
        for index, tr in enumerate(tr_lst):
            td_lst = tr.find_all("td")
            row = [td.get_text().strip().replace(',', '') for td in td_lst]
            # print(index," : ", data)
            
            ps_row = pd.Series(row) #, index=cols)
            pd_temp = pd.concat([pd_temp, ps_row], axis = 1)
        
        
        sd4 = pd_temp
        
        # nsd4 = None
        
        nsd4 = sd4.iloc[1:,:]
        nsd4.columns = sd4.iloc[0,:]
        
        t_EX = ['JPY', 'EUR', 'GBP', 'CAD', 'CHF', 'HKD','THB', 'IDR', 'SEK', 'AUD', 'DKK', 'NOK', 'SAR', 'KWD', 'BHD', 'AED', 'SGD', 'MYR', 'NZD', 'TWD', 'PHP', 'VND', 'PLN', 'RUB', 'ZAR', 'PKR', 'BDT']
        
        try:
        
            nsd4 = nsd4.drop(t_EX, axis=1)
            nsd4 = nsd4.drop([1,2,3,4,5,6,7,8,10], axis=0)
            
            nsd4['Date'] = input_date
            
            nsd4 = nsd4.astype({'Date':'datetime64'})
            nsd4.set_index('Date', inplace=True)
                
            nsd4 = nsd4.astype('float')
            # nsd4.info()
            
            Data_chk = 2
            
            
            
            
        except:
            # 데이터 읽는 것에 에러가 나면 빈 df 지정
            nsd4 = []
            Data_chk = 1
            
            i_count += 1
            
            
        if Data_chk == 2:
            nsd4['CNY_2'] = nsd4['USD'] / nsd4['CNY']
            nsd4['INR_2'] = nsd4['INR'] / nsd4['USD']
            print(nsd4)
            print()
            
            # write_file(f_writer, 4, nsd4)
            return nsd4
    
# =============================================================================
# 0  :  ['USD', '미국 달러', '1,420.60', '1,393.40', '1,431.62', '1.750%', '1,382.38', '1.750%', '1,407.00', '1,408.00', '1.0000']
# 1  :  ['JPY', '일본 100엔', '997.95', '978.79', '1,005.66', '1.750%', '971.08', '1.750%', '988.37', '989.15', '0.7025']
# 2  :  ['EUR', '유럽연합 유로', '1,398.05', '1,370.37', '1,411.47', '1.970%', '1,356.95', '1.970%', '1,384.21', '1,385.33', '0.9838']
# 3  :  ['GBP', '영국 파운드', '1,600.26', '1,568.58', '1,615.63', '1.970%', '1,553.21', '1.970%', '1,584.42', '1,585.27', '1.1261']
# 4  :  ['CAD', '캐나다 달러', '1,054.08', '1,033.22', '1,064.20', '1.970%', '1,023.10', '1.970%', '1,043.65', '1,044.39', '0.7418']
# 5  :  ['CHF', '스위스 프랑', '1,454.97', '1,426.17', '1,468.94', '1.970%', '1,412.20', '1.970%', '1,440.57', '1,441.59', '1.0239']
# 6  :  ['HKD', '홍콩 달러', '181.04', '177.46', '182.78', '1.970%', '175.72', '1.970%', '179.25', '179.38', '0.1274']
# 7  :  ['CNY', '중국 위안', '200.69', '196.73', '208.64', '5.000%', '188.78', '5.000%', '198.71', '198.45', '0.1412']
# 8  :  ['THB', '태국 바트', '38.01', '37.27', '38.39', '2.000%', '36.89', '2.000%', '37.64', '37.67', '0.0268']
# 9  :  ['IDR', '인도네시아 100루피아', '9.46', '9.28', '9.93', '6.000%', '8.72', '7.000%', '9.37', '9.38', '0.0067']
# =============================================================================

#------------------------------------------------------------------------------------------------------------------------------

    

# nsd4 출력 =============================================================================
# 0               USD    CNY
# Date                      
# 2022-09-26 1,408.40 198.40
# =============================================================================



# S5 모듈 - 스틸데일리 --------------------------------------------------------------------------------------------------------
def S5_data():
    print("[S5 - 스틸데일리]")
    
    browser.get("https://www.steeldaily.co.kr/")       #크롬 드라이버에 url 주소 넣고 실행
    # browser.execute_script("window.open('https://www.steeldaily.co.kr/', '_blank');")       #크롬 드라이버에 url 주소 넣고 실행
    # time.sleep(2)
    
    # 로긴 처리 ------------------------------------------------------------------------------------------------------------
    browser.find_element(By.XPATH, '//*[@id="userLogin"]/a[2]').click()    # 로긴 클릭
    
    # 페이지 로딩까지 기다림
    try:
        elem = WebDriverWait(browser, 15).until(EC.presence_of_element_located((By.ID, "user_id")))
        # 성공했을 때 동작 수행    
        # print(elem.text) # 첫번째 결과 출력
    except:
        print("fail to find next page")
        # browser.quit()
        
    elem = browser.find_element(By.ID, 'user_id')
    elem.clear()
    elem.send_keys("ssiron")
    
    elem = browser.find_element(By.ID, 'user_pw')
    elem.clear()
    elem.send_keys("iron0511")
    
    browser.find_element(By.XPATH, '//*[@id="loginForm"]/button').click()    # 조회 클릭
    time.sleep(2)
    
    browser.get("https://www.steeldaily.co.kr/dbcenter/")     # 페이지 이동 - DB
    time.sleep(2)
    
    
    
    # iframe 가져오기
    iframe = browser.find_element(By.XPATH, '//*[@id="db-center-wrap"]/iframe') # id가 mainFrame이라는 요소를 찾아내고 -> iframe임
    browser.switch_to.frame(iframe) # 이 iframe이라는 요소로 focus한다.


    # 5-1) 열연 파트 ----------------------------------------------------------------------------------------------------------
    print("[5-1 열연]")
    browser.find_element(By.XPATH, '//*[@id="page-main"]/div/div/div/div/div[1]/div[1]/ul/li[2]/a').click()  # 열연(HR)
    time.sleep(2)
    soup = BeautifulSoup(browser.page_source, "lxml")
    
    
    tr_lst = soup.find("div", attrs={"id":"print-box"}).find_all("tr", limit=5)   #.find_all("td")  #.find("tr").find_all("td")
    
    # tr_lst = soup.find_all("table")
    # print(td_lst)
    
    # 데이터 변환 시작
    pd_temp = pd.DataFrame()
    cols = ['Date', 'P_SS275', 'P_SM355', 'P_GS', 'H_SS275', 'I_SS275', 'I_SS355']
    
    
    for index, tr in enumerate(tr_lst):
        td_lst = tr.find_all("td")
        row = [td.get_text().strip() for td in td_lst]
        
        # datas = [data.rstrip().strip() for data in datas]
        row = [re.sub(r"[^0-9- ]", "", data).strip() for data in row]
        
        n_row = []
        for data in row:
            temp = data.find(" ")
            if temp > -1:
                data = data[:temp]
            else:
                data = data
            
            n_row.append(data)
            # print(n_row)
        
        ps_row = pd.Series(n_row) #, index=cols)
        # print(ps_row)
        pd_temp = pd.concat([pd_temp, ps_row], axis = 1)
        
        
    sd5_1 = pd_temp.T
    sd5_1.columns = cols
    
    sd5_1 = sd5_1.astype({'Date':'datetime64'})
    sd5_1.set_index('Date', inplace=True)
    sd5_1 = sd5_1.astype('float')
    
    nsd5_1 = sd5_1.dropna(axis=0)
    
    print(nsd5_1)
    print()
    
    # write_file(f_writer, "5_1", nsd5_1)
    #------------------------------------------------------------------------------------------------------------------------------
    
    
    
    # 5-2) GI 파트 ----------------------------------------------------------------------------------------------------------
    print("[5-2 GI]")
    browser.find_element(By.XPATH, '//*[@id="page-main"]/div/div/div/div/div[1]/div[1]/ul/li[5]/a').click()  # GI
    time.sleep(5)
    soup = BeautifulSoup(browser.page_source, "lxml")
        
    tr_lst = soup.find("div", attrs={"id":"print-box"}).find_all("tr", limit=5)   #.find_all("td")  #.find("tr").find_all("td")
    
    
    # 데이터 변환 시작
    pd_temp = pd.DataFrame()
    cols = ['Date', 'UT_Price', 'I_UT_Price']
    
    
    for index, tr in enumerate(tr_lst):
        td_lst = tr.find_all("td")
        row = [td.get_text().strip() for td in td_lst]
        
        row = [re.sub(r"[^0-9- ]", "", data).strip() for data in row]
        
        n_row = []
        for data in row:
            temp = data.find(" ")
            if temp > -1:
                data = data[:temp]
            else:
                data = data
            
            n_row.append(data)
            # print(n_row)
        
        ps_row = pd.Series(n_row) #, index=cols)
        # print(ps_row)
        
        pd_temp = pd.concat([pd_temp, ps_row], axis = 1)
        
        
    sd5_2 = pd_temp.T
    sd5_2.columns = cols
    
    sd5_2 = sd5_2.astype({'Date':'datetime64'})
    sd5_2.set_index('Date', inplace=True)
    sd5_2 = sd5_2.astype('float')
    
    nsd5_2 = sd5_2.dropna(axis=0)
    
    print(nsd5_2)
    print()
    
    # write_file(f_writer, "5_2", nsd5_2)
    
    return nsd5_1, nsd5_2
    
# =============================================================================
# [5-1 열연]
# index :  0
# index :  1
# 2022-09-16
# 1100          50
# 1240   -       0
# 1100          110
# 1110          50
# 1050          120
# 1110          120
# index :  2
# 2022-09-09
# 1050   -       0
# 1240   -       0
# 990   -       0
# 1060   -       0
# 930   -       0
# 990   -       0
# 
# [5-2 GI]
# index :  0
# index :  1
# 2022-09-16
# 1150   -       0
# 1120   -       0
# index :  2
# 2022-09-09
# 1150   -       0
# 1120   -       0
# =============================================================================
#------------------------------------------------------------------------------------------------------------------------------


    print()

# nsd5_1 출력 =============================================================================
#            P_SS275 P_SM355  P_GS H_SS275 I_SS275 I_SS355
# Date                                                    
# 2022-09-23    1200    1260  1200    1200    1150    1210
# 2022-09-16    1100    1240  1100    1110    1050    1110
# 2022-09-09    1050    1240   990    1060     930     990
# 2022-09-02    1050    1240   990    1060     930     990
# =============================================================================

# nsd5_2 출력 =============================================================================
#            UT_Price I_UT_Price
# Date                          
# 2022-09-23     1150       1120
# 2022-09-16     1150       1120
# 2022-09-09     1150       1120
# 2022-09-02     1150       1120
# =============================================================================





# S6 모듈 - iframe 처리 ---------------------------------------------------------------------------------------------------------------------
def S6_data(Input_Date, max_try):    # 날짜를 입력받게 함
    print("[S6 - 상해선물]")
    
    Data_chk = 0     # loop 용 (while) : 0 처음, 1 이전일 체크, 2 완료
    i_count = 0      # loop max count 목적
    
    # 기본 1회 이동
    # browser.get("https://www.shfe.com.cn/")       #크롬 드라이버에 url 주소 넣고 실행
    # time.sleep(3)
    
    browser.get("https://www.shfe.com.cn/en/MarketData/dataview.html?paramid=delaymarket_rb_en")       #크롬 드라이버에 url 주소 넣고 실행
    # browser.execute_script("window.open('https://www.shfe.com.cn/en/MarketData/dataview.html?paramid=delaymarket_rb_en', '_blank');")       #크롬 드라이버에 url 주소 넣고 실행
    time.sleep(5)
        
    browser.find_element(By.XPATH, '//*[@id="kx"]').click()   # Daily express
    time.sleep(5)
    
    
    while (Data_chk < 2 and i_count < max_try):
        
        if Data_chk == 0:
            #처음 날짜 입력 받음
            input_date = Input_Date  #"2022-10-11" #           
            
        else:   # Data_chk == 1 일때
            input_date = datetime.datetime.strptime(input_date, '%Y-%m-%d')

            input_date = input_date+datetime.timedelta(days=-1)
            
            input_date = input_date.strftime("%Y-%m-%d")   # 전날을 string 변환

    
        # 6-1) HC 데이터 가져오기
        # print(input_date)
        
        
        #날짜 입력 받음
        t_date = input_date.split("-")
        t_year, t_mon, t_day = (t_date[0], t_date[1].lstrip("0"), t_date[2].lstrip("0"))
        
        
        # 날짜 선택하는 부분이 여기 들어와야 할 것 - year는 일단 제외 (포함 필요)
        browser.find_element(By.XPATH, f'//*[@id="calendar"]/div/div/div/select[2]/option[{t_mon}]').click()   # 월 
        time.sleep(2)
        browser.find_element(By.LINK_TEXT, t_day).click()      # 일
        time.sleep(6)
        
        
        soup = BeautifulSoup(browser.page_source, "lxml")
        
        
        # 데이터 변환 시작
        pd_temp = pd.DataFrame()
        # cols = ['Date', 'BDI', 'BCI', 'BPI', 'BSI']
        
        
        tr1_lst = soup.find_all("tr", attrs={"class":"pinz"}, limit=20)  #.find_all("tbody", limit=5) #.find_all("td")  #.find("tr").find_all("td")
        
        
        # temp = 0 
        
        # for index, tr1 in enumerate(tr1_lst):
        #     # str_lst = tr.find_all("strong")
        #     # try:
        #         if tr1.strong.get_text().strip() in "Species:hc_f":  #"Species:rb_f":
        #             # print(tr1.find("strong"))
        #             # print(tr.parent.next_sibling)
        #             tr2_lst = tr1.parent.parent.find_all("tr")
                    
        #             temp = index
                    
        #             for idx, tr2 in enumerate(tr2_lst):
        #                 td_lst = tr2.find_all("td")
        #                 row = [td.get_text().strip() for td in td_lst]
                    
        #                 ps_row = pd.Series(row) #, index=cols)
        #                 pd_temp = pd.concat([pd_temp, ps_row], axis = 1)
        #     # except:
        #     #     pass    
            
        
        
        
        # tr_lst.strong.get_text()
        for index, tr1 in enumerate(tr1_lst):
            # str_lst = tr.find_all("strong")
            try:
                if tr1.strong.get_text().strip() in "Species:hc_f":  #"Species:rb_f":
                    # print(tr1.find("strong"))
                    # print(tr.parent.next_sibling)
                    tr2_lst = tr1.parent.parent.find_all("tr")
                    
                    for idx, tr2 in enumerate(tr2_lst):
                        td_lst = tr2.find_all("td")
                        row = [td.get_text().strip() for td in td_lst]
                    
                        ps_row = pd.Series(row) #, index=cols)
                        pd_temp = pd.concat([pd_temp, ps_row], axis = 1)
            except:
                pass            
                
        
        try: 
            sd6_1 = pd_temp.T
            
            nsd6_1 = sd6_1[5:6]  # 4번째 선택
            nsd6_1.columns = sd6_1.iloc[0,:]
            
            
            nsd6_1['Date'] = input_date
            
            nsd6_1 = nsd6_1.astype({'Date':'datetime64'})
                        
            
            nsd6_1.set_index('Date', inplace=True)
            nsd6_1 = nsd6_1.astype('float')
            
            # nsd6_1.columns
            # nsd6_1.dropna(['Settle','ch1','ch2','Volume','Turnover','O.I & Change'], axis=1)
            nsd6_1 = nsd6_1.drop(nsd6_1.columns[6:], axis=1)
            
            Data_chk = 2
            
        
        except:
            # 데이터가 없을 경우
            nsd6_1 = []
            
            Data_chk = 1            
            i_count += 1
        
        
        if Data_chk == 2:   # 잘 나왔으면 6-2)값도 같이 가져오도록 함
            # 우선 6-1) 출력해놓음
            print("[6-1 - HC]")
            print(nsd6_1)
            print()
            
            # write_file(f_writer, "6_1", nsd6_1)
        
        
            # 6-2) Rebar 데이터 가져오기 - 위에서 결정된 input_date를 그대로 사용
            print("[6-2 - Rebar]")
            
            # 데이터 변환 시작
            pd_temp = pd.DataFrame()
            # cols = ['Date', 'BDI', 'BCI', 'BPI', 'BSI']
            
            
            # tr1_lst = soup.find_all("tr", attrs={"class":"pinz"})  #.find_all("tbody", limit=5) #.find_all("td")  #.find("tr").find_all("td")
        
            # tr_lst.strong.get_text()
            for index, tr1 in enumerate(tr1_lst):
                # str_lst = tr.find_all("strong")
                try:
                    if tr1.strong.get_text().strip() in "Species:rb_f":  #"Species:rb_f":
                        # print(tr1.find("strong"))
                        # print(tr.parent.next_sibling)
                        tr2_lst = tr1.parent.parent.find_all("tr")
                        
                        for idx, tr2 in enumerate(tr2_lst):
                            td_lst = tr2.find_all("td")
                            row = [td.get_text().strip() for td in td_lst]
                        
                            ps_row = pd.Series(row) #, index=cols)
                            pd_temp = pd.concat([pd_temp, ps_row], axis = 1)
                except:
                    pass            
                    
            
            try:
                sd6_2 = pd_temp.T
                
                nsd6_2 = sd6_2[5:6]   # 4번째 선택
                nsd6_2.columns = sd6_2.iloc[0,:]
                
                
                nsd6_2['Date'] = input_date
                
                nsd6_2 = nsd6_2.astype({'Date':'datetime64'})
                
                
                nsd6_2.set_index('Date', inplace=True)
                nsd6_2 = nsd6_2.astype('float')
                
                # nsd6_1.columns
                # nsd6_1.dropna(['Settle','ch1','ch2','Volume','Turnover','O.I & Change'], axis=1)
                nsd6_2 = nsd6_2.drop(nsd6_2.columns[6:], axis=1)
            
            except:
                # 데이터가 없을 경우
                nsd6_2 = []
                
            
            print(nsd6_2)
            print()
            
            # write_file(f_writer, "6_2", nsd6_2)
            
            return nsd6_1, nsd6_2
    
    
# nsd6_1 출력 =============================================================================
# 0          Delivery month Pre settle  Open  High   Low Close
# Date                                                        
# 2022-08-12           2210       4080  4118  4153  4100  4150
# =============================================================================
    
# nsd6_2 출력 =============================================================================
# 0          Delivery month Pre settle  Open  High   Low Close
# Date                                                        
# 2022-08-12           2210       4080  4118  4153  4100  4150
# =============================================================================
    
    


# S7 모듈 : 로긴, 자주 접속하면 block 처리됨, 조심!!------------------------------------------------------------------------------------------
def S7_data():
    
    print("[S7 - Argus]")
    
    browser.get("https://myaccount.argusmedia.com/login")       #크롬 드라이버에 url 주소 넣고 실행
    # browser.execute_script("window.open('https://myaccount.argusmedia.com/login', '_blank');")       #크롬 드라이버에 url 주소 넣고 실행
    time.sleep(2)
    
    # 로긴 처리 ------------------------------------------------------------------------------------------------------------
    elem = browser.find_element(By.ID, 'username')
    elem.clear()
    elem.send_keys("miso.suh@samsung.com")
    
    elem = browser.find_element(By.ID, 'password')
    elem.clear()
    elem.send_keys("qwe123!")
    
    browser.find_element(By.XPATH, '/html/body/app-root/app-public/div[2]/div/app-login/div/div/form/div[2]/div/button').click()    # 조회 클릭
    time.sleep(7)
    
    browser.get("https://metals.argusmedia.com/metal/base-metals-nickel")     # 페이지 이동 - DB
    time.sleep(7)
    
    
    soup = BeautifulSoup(browser.page_source, "lxml")
    # browser.quit()
    
    
    # 데이터 변환 시작
    pd_temp = pd.DataFrame()
    # cols = ['Date', 'BDI', 'BCI', 'BPI', 'BSI']
    
    
    tr_lst = soup.find("tbody", attrs={"id":"collapse-base-metals-nickel-lme"}).find_all("tr", attrs={"class":"searchable"}) #.find_all("td")  #.find("tr").find_all("td")
    # print(tr_lst)
    
    
    for index, tr in enumerate(tr_lst):
        td_lst = tr.find_all("td")
        row  = [td.get_text().strip().replace(',', '') for td in td_lst]
        # print(index," : ", data)
        
        ps_row = pd.Series(row)  #, index=cols)
        pd_temp = pd.concat([pd_temp, ps_row], axis = 1)
        
        
    sd7 = pd_temp.T
    # print(sd7)  # 전체 테이블
    
    sd7 = sd7.drop([1,2,3,5,6,7,9], axis=1)   # 쓸데없는 열 지우고
    
    
    ## (중요 메모!!)  여기에다 cash official 값으로 옆에 있는 값을 반환하도록 해야 할 것 같음, 데이터프레임의 구조가 바뀌는 듯 함
    
    
    t_sd7 = sd7.reset_index()
    t_sd7 = t_sd7.loc[[0, 7]]
    
    t_date = t_sd7.iloc[0, 3]
    t_sd7 = t_sd7.drop(["index", 8], axis=1)   # 쓸데없는 열 지우고
    
    tt_sd7 = t_sd7.T
    tt_sd7.columns = tt_sd7.iloc[0,:]
    nsd7 = tt_sd7.iloc[[1],:]
    
    nsd7['Date'] = t_date
    nsd7 = nsd7.astype({'Date':'datetime64'})
    
    nsd7.set_index('Date', inplace=True)
        
    nsd7 = nsd7.astype('float')
    
    print(nsd7)
    
    # 로그아웃 -> 이걸 하면 로봇 작동으로 블락되어 버림
    # browser.find_element(By.XPATH, '//*[@id="settingsButton"]/div').click()
    # time.sleep(2)
    # browser.find_element(By.XPATH, '//*[@id="signoutButton"]').click()
    
    
    print()
    
    
    # write_file(f_writer, "7", nsd7)
    
    return nsd7
    
# nsd7 출력=============================================================================
# 0           LME Nickel Cash Official  LME Nickel Warehouse Stocks
# Date                                                             
# 2022-09-27                 21,882.50                    51,864.00
# =============================================================================




# S8 모듈 ---------------------------------------------------------------------------------------------------------------------
def S8_data():
    print("[S8 - CUSteel]")

    browser.get("https://www.custeel.com/")       #크롬 드라이버에 url 주소 넣고 실행
    # browser.execute_script("window.open('https://www.custeel.com/', '_blank');")       #크롬 드라이버에 url 주소 넣고 실행
    time.sleep(2)
    
    browser.find_element(By.XPATH, '//*[@id="logoutli"]/a').click()    # 조회 클릭
    # time.sleep(1)
    
    
    # 로긴 처리 ------------------------------------------------------------------------------------------------------------
    elem = browser.find_element(By.ID, 'username')
    elem.clear()
    elem.send_keys("4206")
    
    elem = browser.find_element(By.ID, 'password')
    elem.clear()
    elem.send_keys("4206")
    
    browser.find_element(By.ID, 'loginBtn').click()    # 조회 클릭
    time.sleep(3)
    
    
    # 8-1. 재고 현황 ------------------------------------------------------------------------------------------------------------
    print("[8-1 재고 현황]")
    
    browser.get("http://www.custeel.com/shouye/common/viewArticle.jsp?group=1001&cat=1002006&articleID=6869519")
    time.sleep(3)
    
    browser.find_element(By.XPATH, '/html/body/table/tbody/tr[1]/td[1]/div[2]/ul/li[1]/a').click()  # 최신 이동
    
    # browser.to_switch(browser.window_handles[1])  
    browser.switch_to.window(browser.window_handles[-1])
    browser.get_window_position(browser.window_handles[-1])
    
        
    soup = BeautifulSoup(browser.page_source, "lxml")
    # browser.quit()
    
    tr_lst = soup.find("div", attrs={"id":"main_c"}).find("tbody").find_all("tr") #.find_all("td")  #.find("tr").find_all("td")
    # print(tr_lst)
    
    
    # 데이터 변환 시작
    pd_temp = pd.DataFrame()
    # cols = ['Date', 'BDI', 'BCI', 'BPI', 'BSI']
    
    
    for index, tr in enumerate(tr_lst):
        td_lst = tr.find_all("td")
        row = [td.get_text().strip() for td in td_lst]
        # print(index," : ", data)
        
        ps_row = pd.Series(row) #, index=cols)
        pd_temp = pd.concat([pd_temp, ps_row], axis = 1)
        
        
    sd8_1 = pd_temp
    sd8_1 = sd8_1.loc[[1,2,4]]
    
    cols = ['Date', '철근', '열연', '냉연', '선재', '후판', '합계']
    sd8_1.columns = cols
    
    sd8_1 = sd8_1.astype({'Date':'datetime64'})
    sd8_1.set_index('Date', inplace=True)
        
    sd8_1 = sd8_1.astype('float')
    
    
    sd8_1['flat재고'] = sd8_1['열연'] + sd8_1['냉연'] + sd8_1['후판']
    sd8_1['long재고'] = sd8_1['철근'] + sd8_1['선재']
    
    nsd8_1 = sd8_1.drop("합계", axis=1)   # 쓸데없는 열 지우고
    
    print(nsd8_1)
    print()
    
    # write_file(f_writer, "8_1", nsd8_1)
    
    
    # 8-2. 62분광 ------------------------------------------------------------------------------------------------------------
    print("[8-2 62분광]")
    
    browser.get("http://index.custeel.com/")
    time.sleep(3)
    
    # iframe 가져오기
    iframe = browser.find_element(By.XPATH, '//*[@id="iframe"]/iframe') # id가 mainFrame이라는 요소를 찾아내고 -> iframe임
    browser.switch_to.frame(iframe) # 이 iframe이라는 요소로 focus한다.
    
   
    # 체크박스 선택
    browser.find_element(By.XPATH, '/html/body/div[2]/div[1]/div[4]/input[1]').click()    # 조회 클릭
    browser.find_element(By.XPATH, '/html/body/div[2]/div[1]/div[4]/input[2]').click()    # 조회 클릭
    browser.find_element(By.XPATH, '/html/body/div[2]/div[1]/div[4]/input[3]').click()    # 조회 클릭


    browser.find_element(By.XPATH, '/html/body/div[2]/div[1]/div[5]/a').click()    # 하단 메뉴
    browser.find_element(By.XPATH, '/html/body/div[2]/div[1]/div[6]/input[4]').click()    # 체크박스
    
    soup = BeautifulSoup(browser.page_source, "lxml")
    # browser.quit()
    
    
    # 데이터 변환 시작
    pd_temp = pd.DataFrame()
    # cols = ['Date', 'BDI', 'BCI', 'BPI', 'BSI']  
    
    
    tr_lst = soup.find("div", attrs={"id":"info"}).find("tbody").find_all("tr", limit=5) #.find_all("td")  #.find("tr").find_all("td")
    # print(tr_lst)
    
    for index, tr in enumerate(tr_lst):
        td_lst = tr.find_all("td")
        row = [td.get_text().strip() for td in td_lst]
        # print(index," : ", data)
        
        ps_row = pd.Series(row)  #, index=cols)
        pd_temp = pd.concat([pd_temp, ps_row], axis = 1)
        
        
    sd8_2 = pd_temp.T
    
    sd8_2 = sd8_2.drop(range(2,8), axis=1)   # 쓸데없는 열 지우고
    
    t_sd8_2 = sd8_2.reset_index()
    t_sd8_2 = t_sd8_2.loc[[1,2,3,4]]           # 필요한 행 선택
    
    t_sd8_2 = t_sd8_2.drop("index", axis=1)   # 쓸데없는 열 지우고
    
    t_sd8_2.columns = ["Date", "수입62분광(USD)"]
    
    nsd8_2 = t_sd8_2.astype({'Date':'datetime64'})
    
    nsd8_2.set_index('Date', inplace=True)
        
    nsd8_2 = nsd8_2.astype('float')
    
    
    print(nsd8_2)
    print()
    
    # write_file(f_writer, "8_2", nsd8_2)
    
    
    # 8-3. 청도항 62분광 ------------------------------------------------------------------------------------------------------------
    print("[8-3 청도항 62분광]")
    
    #체크박스
    browser.find_element(By.XPATH, '/html/body/div[2]/div[1]/div[6]/input[4]').click()    # 체크박스
    browser.find_element(By.XPATH, '/html/body/div[2]/div[1]/div[6]/input[13]').click()    # 체크박스
    
    soup = BeautifulSoup(browser.page_source, "lxml")
    # browser.quit()
    
    
    # 데이터 변환 시작
    pd_temp = pd.DataFrame()
    # cols = ['Date', 'BDI', 'BCI', 'BPI', 'BSI']  
    
    
    tr_lst = soup.find("div", attrs={"id":"info"}).find("tbody").find_all("tr", limit=5) #.find_all("td")  #.find("tr").find_all("td")
    # print(tr_lst)
    
    for index, tr in enumerate(tr_lst):
        td_lst = tr.find_all("td")
        row = [td.get_text().strip() for td in td_lst]
        # print(index," : ", data)
        
        ps_row = pd.Series(row)  #, index=cols)
        pd_temp = pd.concat([pd_temp, ps_row], axis = 1)
        
        
    sd8_3 = pd_temp.T
    
    sd8_3 = sd8_3.drop(range(2,8), axis=1)   # 쓸데없는 열 지우고
    
    t_sd8_3 = sd8_3.reset_index()
    t_sd8_3 = t_sd8_3.loc[[1,2,3,4]]           # 필요한 행 선택
    
    t_sd8_3 = t_sd8_3.drop("index", axis=1)   # 쓸데없는 열 지우고
    
    t_sd8_3.columns = ["Date", "청도62분광(RMB)"]
    
    nsd8_3 = t_sd8_3.astype({'Date':'datetime64'})
    
    nsd8_3.set_index('Date', inplace=True)
        
    nsd8_3 = nsd8_3.astype('float')
    
    
    print(nsd8_3)
    print()
    
    # write_file(f_writer, "8_3", nsd8_3)
    
    
    # 8-4. 상해 HRC ------------------------------------------------------------------------------------------------------------
    print("[8-4 상해 HRC]")
    
    browser.get("http://www.custeel.com/shouye/common/viewArticle.jsp?articleID=6599445")
    time.sleep(2)
    
    browser.find_element(By.XPATH, '/html/body/table/tbody/tr[1]/td[1]/div[2]/ul/li[1]/a').click()    # 클릭
    time.sleep(2)
    
    # soup = BeautifulSoup(browser.page_source, "lxml")
    
    browser.switch_to.window(browser.window_handles[-1])
    browser.get_window_position(browser.window_handles[-1])
    
    t_date = browser.find_element(By.XPATH, '/html/body/table/tbody/tr[1]/td[1]/div[1]/div[2]').text[:10]   # 날짜 추출 / '2021-07-15'
    
    t_value = browser.find_element(By.XPATH, '/html/body/table/tbody/tr[1]/td[1]/div[1]/div[4]/table/tbody/tr[12]/td[5]').text   # 값 추출 / '5960'
    
    t_sd8_4 = pd.DataFrame(index=range(1,2), columns={'Date','SHA_HRC'})
    t_sd8_4['Date'] = t_date
    t_sd8_4['SHA_HRC'] = t_value
    
    nsd8_4 = t_sd8_4.astype({'Date':'datetime64'})
    
    nsd8_4.set_index('Date', inplace=True)
        
    nsd8_4 = nsd8_4.astype('float')
    
    print(nsd8_4)
    print()
    
    
    
    
    # 8-5. 천진 HRC ------------------------------------------------------------------------------------------------------------
    print("[8-5 천진 HRC]")
    
    browser.get("http://www.custeel.com/shouye/common/viewArticle.jsp?articleID=6599443")
    time.sleep(2)
    
    browser.find_element(By.XPATH, '/html/body/table/tbody/tr[1]/td[1]/div[2]/ul/li[1]/a').click()    # 클릭
    time.sleep(2)
    

    # soup = BeautifulSoup(browser.page_source, "lxml")
    
    browser.switch_to.window(browser.window_handles[-1])
    browser.get_window_position(browser.window_handles[-1])
    
    t_date = browser.find_element(By.XPATH, '/html/body/table/tbody/tr[1]/td[1]/div[1]/div[2]').text[:10]   # 날짜 추출 / '2021-07-15'
    # /html/body/table/tbody/tr[1]/td[1]/div[1]/div[2]
    t_value = browser.find_element(By.XPATH, '/html/body/table/tbody/tr[1]/td[1]/div[1]/div[4]/table/tbody/tr[10]/td[5]').text   # 값 추출 / '5960'
    
    
    
    
    t_sd8_5 = pd.DataFrame(index=range(1,2), columns={'Date','TJ_HRC'})
    t_sd8_5['Date'] = t_date
    t_sd8_5['TJ_HRC'] = t_value
    
    nsd8_5 = t_sd8_5.astype({'Date':'datetime64'})
    
    nsd8_5.set_index('Date', inplace=True)
        
    nsd8_5 = nsd8_5.astype('float')
    
    print(nsd8_5)
    print()
    
    
    
    # 8-6. 상해 GI ------------------------------------------------------------------------------------------------------------
    print("[8-6 상해 GI]")
    
    browser.get("https://www.custeel.com/shouye/common/viewArticle.jsp?group=1001007002&articleID=7022215")
    time.sleep(2)
    
    browser.find_element(By.XPATH, '/html/body/table/tbody/tr[1]/td[1]/div[2]/ul/li[1]/a').click()    # 클릭
    time.sleep(2)

    # soup = BeautifulSoup(browser.page_source, "lxml")
    
    browser.switch_to.window(browser.window_handles[-1])
    browser.get_window_position(browser.window_handles[-1])
    
    t_date = browser.find_element(By.XPATH, '/html/body/table/tbody/tr[1]/td[1]/div[1]/div[2]').text[:10]   # 날짜 추출 / '2021-07-15'
    # /html/body/table/tbody/tr[1]/td[1]/div[1]/div[2]
    t_value = browser.find_element(By.XPATH, '/html/body/table/tbody/tr[1]/td[1]/div[1]/div[4]/table/tbody/tr[18]/td[5]').text   # 값 추출 / '5960'
    
    
    
    
    t_sd8_6 = pd.DataFrame(index=range(1,2), columns={'Date','SHA_GI'})
    t_sd8_6['Date'] = t_date
    t_sd8_6['SHA_GI'] = t_value
    
    nsd8_6 = t_sd8_6.astype({'Date':'datetime64'})
    
    nsd8_6.set_index('Date', inplace=True)
        
    nsd8_6 = nsd8_6.astype('float')
    
    print(nsd8_6)
    print()
    
    
    
    return nsd8_1, nsd8_2, nsd8_3, nsd8_4, nsd8_5, nsd8_6



# nsd8_1 출력=============================================================================
#                철근     열연     냉연     선재     후판  flat재고  long재고
# Date                                                         
# 2022-09-23 485.90 240.20 130.10 117.90 112.50  370.30  603.80
# 2022-09-16 496.30 241.60 132.00 122.40 113.10  373.60  618.70
# 2022-08-26 508.30 256.40 134.40 119.20 120.30  390.80  627.50
# =============================================================================

# nsd8_2 출력=============================================================================
#             수입62분광(USD)
# Date                   
# 2022-09-27        94.78
# 2022-09-26        93.59
# 2022-09-23        95.05
# 2022-09-22        95.58
# =============================================================================

# nsd8_3 출력=============================================================================
#             청도62분광(RMB)
# Date                   
# 2022-09-27       760.00
# 2022-09-26       750.00
# 2022-09-23       757.00
# 2022-09-22       755.00
# =============================================================================

    
#------------------------------------------------------------------------------------------------------------------------------



# S9 모듈 ---------------------------------------------------------------------------------------------------------------------
def S9_data(Input_Date, max_try):
    print("[S9 - MYSteel]")

      
    # 9-1. 원료탄 ------------------------------------------------------------------------------------------------------------
    print("[9-1 원료탄]")
    
    browser.get("https://index.mysteel.com/xpic/detail.html?tabName=jiaotan")    
    # browser.execute_script("window.open('https://index.mysteel.com/xpic/detail.html?tabName=jiaotan', '_blank');")       #크롬 드라이버에 url 주소 넣고 실행
    time.sleep(2)
    
    soup = BeautifulSoup(browser.page_source, "lxml")
    # browser.quit()
    
    tr_lst = soup.find("table", attrs={"class":"detailTab"}).find_all("tr") #.find_all("td")  #.find("tr").find_all("td")
    # print(tr_lst)
    
    
    # 데이터 변환 시작
    pd_temp = pd.DataFrame()
    # cols = ['Date', 'BDI', 'BCI', 'BPI', 'BSI']  
    
    
    for index, tr in enumerate(tr_lst):
        td_lst = tr.find_all("td")
        row = [td.get_text().strip() for td in td_lst]
        # print(index," : ", data)
        
        ps_row = pd.Series(row)  #, index=cols)
        pd_temp = pd.concat([pd_temp, ps_row], axis = 1)
        
        
    sd9_1 = pd_temp.T
    
    temp = sd9_1.iloc[0,0]   #당일 날짜
    t_date = datetime.datetime.strptime(temp,'(%Y%m%d)')        # 당일
    p_date = t_date - datetime.timedelta(days=1)                # 전일
    
    t_sd9_1 = pd.DataFrame(index=range(1,3), columns={'Date','원료탄'})
    t_sd9_1['Date'] = [t_date, p_date]
    t_sd9_1['원료탄'] = [sd9_1.iloc[2,1], sd9_1.iloc[2,2]]
    
    nsd9_1 = t_sd9_1.astype({'Date':'datetime64'})
    
    nsd9_1.set_index('Date', inplace=True)
        
    nsd9_1 = nsd9_1.astype('float')
    
    print(nsd9_1)
    print()
    
    # write_file(f_writer, "9_1", nsd9_1)
    
  
    
    # 9-2. HRC ------------------------------------------------------------------------------------------------------------
    
    Data_chk = 0     # loop 용 (while) : 0 처음, 1 이전일 체크, 2 완료
    i_count = 0      # loop max count 목적
    
    # max_try = 10  input_date = "2022-10-5" 
    while (Data_chk < 2 and i_count < max_try):
        
        if Data_chk == 0:
            #처음 날짜 입력 받음
            input_date = Input_Date  #"2022-10-4"            
            
        else:   # Data_chk == 1 일때
            input_date = datetime.datetime.strptime(input_date, '%Y-%m-%d')

            input_date = input_date+datetime.timedelta(days=-1)
            
            input_date = input_date.strftime("%Y-%m-%d")   # 전날을 string 변환
        
        
        try:
            
            sDate = input_date  #Input_Date  #"2022-10-5"
            # print(sDate)
            
            sOption = 'rzbj'
            url = 'http://api.mysteel.com/dbus/new/pricemap.html?type=0&defaultSign=0&datetime=' + sDate +'&option=' + sOption
            # http://api.mysteel.com/dbus/new/pricemap.html?type=0&defaultSign=0&datetime=2022-09-29&option=rzbj
         
            
            #새로운 시도 --------------------------------------------------------------------------------------------------------
            browser.get(url)     
            
        
            data = browser.page_source
            
            # test = "<sfsrf>erwrewer<sdfsf>"
            k = re.compile('<.{,70}>')
            st = re.sub(k, '', data)
            # print(st)
            
            
            info = json.loads(st)
            
            
            df = json_normalize(info['data'])          # 데이터프레임 변환
            
            # print(df)
            # df.columns
            
            t_mean = df['value'].astype('float').mean()  # 평균값 계산
            
            sd9_2 = pd.DataFrame(index=range(1,2), columns={'Date','HRC avg'})
            sd9_2['Date'] = [sDate]
            sd9_2['HRC avg'] = [t_mean]
            
            nsd9_2 = sd9_2.astype({'Date':'datetime64'})
            
            nsd9_2.set_index('Date', inplace=True)
            nsd9_2 = nsd9_2.astype('float')
            
            # print(nsd9_2)
            
            Data_chk = 2
            
        except:
            # print("fail!! " + str(i_count))
            nsd9_2 = []
            Data_chk = 1
            
            i_count += 1
            
            
    
    # print("Data_chk : " + str(Data_chk))
            
    if Data_chk == 2:    # 데이터 잘 끌어왔을 때
        print("[9-2 HRC avg]")
        print(nsd9_2)
        print()
        
        # write_file(f_writer, "9_2", nsd9_2)
        
        return nsd9_1, nsd9_2
    
    
    # 중간 테스트 =============================================================================
    #     test = "<sfsrf>erwrewer<sdfsf>"
    #     k = re.compile('<.{,5}>')
    #     st = re.sub(k, '', test)
    #     print(st)
    # =============================================================================
    
   
    
#------------------------------------------------------------------------------------------------------------------------------

  
# nsd9_1 출력=============================================================================
#                 원료탄
# Date               
# 2022-09-27 2,666.00
# 2022-09-26 2,666.00
# =============================================================================
    

# nsd9_2 출력=============================================================================
#                 HRC avg
# Date                   
# 2022-09-29  4054.909091
# =============================================================================




# S10 - CRU ---------------------------------------------------------------------------------------------------------------------
def S10_data():
    print("[S10 - CRU]")

      
    browser.get("https://cruonline.crugroup.com/")     
    # browser.execute_script("window.open('https://cruonline.crugroup.com/', '_blank');")       #크롬 드라이버에 url 주소 넣고 실행
    time.sleep(3)
    
    
    # 로긴 처리 ------------------------------------------------------------------------------------------------------------
    try:
        elem = browser.find_element(By.ID, 'username')
        elem.clear()
        elem.send_keys("emanu.fabian@samsung.com")
        
        elem = browser.find_element(By.ID, 'password')
        elem.clear()
        elem.send_keys("SamsungCru2020")
        
        browser.find_element(By.XPATH, '/html/body/div[2]/main/section/div/div/div/form/div[2]/button').click()    # 조회 클릭
        time.sleep(3)
        
    except:
        pass
    
    # Price 페이지 이동
    browser.get("https://cruonline.crugroup.com/prices/Steel%20Sheet%20Products?tab=weekly")   
    time.sleep(7)
    
    soup = BeautifulSoup(browser.page_source, "lxml")
    # browser.quit()
    
    
    # 데이터 변환 시작
    pd_temp = pd.DataFrame()
    
    
    div_lst = soup.find("div", attrs={"comp-id":"707"}).find_all("div", attrs={"comp-id":["780", "784"]})
    len(div_lst)
    
    for index, div in enumerate(div_lst):
        span_lst = div.find_all("span")
        row = [span.get_text().strip().replace(',', '') for span in span_lst]
        # print(index," : ", data)
        
        ps_row = pd.Series(row)  #, index=cols)
        pd_temp = pd.concat([pd_temp, ps_row], axis = 1)
        
    
    
    # Date 가져오기
    s_date = browser.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/main/div/div/div[4]/div[1]/section/div[2]/div/div/div/div[2]/div[2]/div[3]/div[4]/div[6]/div/div/div/div[2]/div[2]/div[1]/div[2]/div/div/div[5]/div[3]/div/span[1]').text
    
    
        
    sd10 = pd_temp   
    t_sd10 = sd10.iloc[[4],:]
    t_sd10.columns = ["HRC US", "HDG US"]
    t_sd10["Date"] = s_date
    
    
    nsd10 = t_sd10.astype({'Date':'datetime64'})
    
    nsd10.set_index('Date', inplace=True)
        
    nsd10 = nsd10.astype('float')
    
    print(nsd10)
    print()
    
    # write_file(f_writer, "10", nsd10)
    
    return nsd10


# nsd10 출력물 =============================================================================
#             HRC US  HDG US
# Date                      
# 2022-09-28   791.0  1121.0
# =============================================================================
    



In [9]:


if __name__ == "__main__":
    
    # options = webdriver.ChromeOptions()
    # options.headless = False    # True  False
    # options.add_argument("window-size=1920x1080")
    # options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36")

    options = webdriver.ChromeOptions()
    options.add_argument('--headless')        # Head-less 설정
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    browser = webdriver.Chrome('chromedriver', options=options)

     
    # browser = webdriver.Chrome('D:/★프로세스 혁신★/22.09월/철강 MI 실행 추진/철강MI_데이터 수집/chromedriver', options=options)      #같은 폴더내 저장,    executable_path='chromedriver')
    #browser.maximize_window()  # 창 최대화


    # 기준일자를 전일로 설정 (실행일 기준 전일)
    Input_Date = datetime.datetime.today() + datetime.timedelta(days=-1)
    Input_Date = Input_Date.strftime('%Y-%m-%d')
    # type(Input_Date)
    
    
    site_lst = ['platts', 'shipping', 'oil', 'ERate', 'steeldaily', 'shfe', 'argus', 'custeel', 'mysteel', 'cru']   # 10개
    # len(site_lst)
    
    str_lst = ['01', '02', '03', '04', '05_1', '05_2', '06_1', '06_2', '07', '08_1', '08_2','08_3', '08_4', '08_5','08_6','09_1','09_2', '10']  # 18개
    # len(str_lst)
    
    
    # log_df = pd.DataFrame(index=['Date'] + list(range(1,11)), columns={'chked'})        # 기존 log - 잘 실행됐는지 여부 체크, 재실행시 안된 것만 체크/실행
        
    
    # temp = '08_3'
    # int(temp[:2])
    
    # 1) 기존 df 읽기 -------------------------------------------------------------------------------------------
    input_file = '/content/drive/MyDrive/2. new_data.xlsx'
    
    log_file = '/content/drive/MyDrive/3. log_file.xlsx'
    
    
    # old df 저장하는 리스트
    odf_lst = {}
    
    for i in str_lst:     # 전체 시트 대상
    
        try:
            name_index = int(i[:2]) - 1
            df = pd.read_excel(input_file, sheet_name= site_lst[name_index] + "_" + i)      #site_lst, str_lst로 조합된 시트 읽어오기
            
            df = df.astype({'Date':'datetime64[ns]'})
            df.set_index('Date', inplace=True)
            df = df.astype('float')
            # df.info()
            
            # odf_lst.append(df)
            odf_lst[i] = df
            
        except:
            odf_lst[i] = pd.DataFrame()    # 기존파일이 없으면 빈 값을 집어넣음
        
    # 기존 파일 이름 바꾸기
    try:
        old_file = '/content/drive/MyDrive/4. old_data.xlsx'
        
        wb = load_workbook(input_file)
        wb.save(old_file)
        wb.close()
    
    except:
        pass
    
    
    # log 파일 읽기-------------------------------------------------------
    # log_file = 'D:/★프로세스 혁신★/22.09월/철강 MI 실행 추진/철강MI_데이터 수집/log_chked.xlsx'
    
    try:
        log_df = pd.read_excel(log_file)
        
        log_df = log_df.astype({'Date':'datetime64[ns]'})
        
        
        log_df.set_index('Unnamed: 0', inplace=True)   # 기준을 index로 정해야 할 듯
        # log_df['Date'][0]
        
        if log_df['Date'][0].strftime('%Y-%m-%d') != Input_Date:   # 날짜가 다르면 초기화
            # df 초기화
            log_df = pd.DataFrame(index=['Date'] + list(range(1,11)), columns={'chked'}).T        # 기존 log - 잘 실행됐는지 여부 체크, 재실행시 안된 것만 체크/실행
            log_df.loc['chked', 'Date'] = Input_Date
            
        
        
        # log_df.columns = ['item', 'chked']
        # log_df.index
        
        # log_df.set_index('Date', inplace=True)   # 기준을 index로 정해야 할 듯
        
        # log_df.columns
        
        # log_df = log_df.iloc[:,1:]
                
        # log_df.loc['Date']['chked']
        
        # input_date = datetime.datetime.strptime(input_date, '%Y-%m-%d')
        
        
    except:    # 파일이 없을 경우
        # df 초기화
        log_df = pd.DataFrame(index=['Date'] + list(range(1,11)), columns={'chked'}).T        # 기존 log - 잘 실행됐는지 여부 체크, 재실행시 안된 것만 체크/실행
        log_df.loc['chked', 'Date'] = Input_Date
        
        
        # log_df.loc['chked', 'Date'] = Input_Date
        
        
    
    # 2) 신규 df 읽기 -------------------------------------------------------------------------------------------
    
    company = False  # True 회사, False 회사 밖
    
    ndf_lst = {}            # new df 저장하는 딕셔너리
    
    o_feedback = "< 실행결과 - 피드백 >\n"
    o_dft = ["미실행 - 이미 updated\n", "실행 - Updated\n", "오류 - 재실행 필요\n"]
    
    
    # 총 10개 모듈 실행
    
    # 1
    o_feedback += "01 - "
    if log_df.loc['chked', 1] != True:
        try:
            ndf_lst['01'] = S1_data(Input_Date, num_days=5, company=True)     # 기본: 최근 5일치 조회
            log_df.loc['chked', 1] = True
            o_feedback += o_dft[1]
        except:
            log_df.loc['chked', 1] = False
            o_feedback += o_dft[2]
    else:
        o_feedback += o_dft[0]
            
    
    # 2
    o_feedback += "02 - "
    if log_df.loc['chked', 2] != True:
        try:
            ndf_lst['02'] = S2_data()
            log_df.loc['chked', 2] = True
            o_feedback += o_dft[1]
        except:
            log_df.loc['chked', 2] = False
            o_feedback += o_dft[2]
    else:
        o_feedback += o_dft[0]
            
    
    # 3
    o_feedback += "03 - "
    if log_df.loc['chked', 3] != True:
        try:
            ndf_lst['03'] = S3_data()
            log_df.loc['chked', 3] = True
            o_feedback += o_dft[1]
        except:
            log_df.loc['chked', 3] = False
            o_feedback += o_dft[2]
    else:
        o_feedback += o_dft[0]
            
            
    # 4
    o_feedback += "04 - "
    if log_df.loc['chked', 4] != True:
        try:
            ndf_lst['04'] = S4_data(Input_Date, max_try=10)      # 환율 조회, 기준일 데이터 없으면 그 전날 찾을 때까지 반복(최대 5일전까지 - max_try)
            log_df.loc['chked', 4] = True
            o_feedback += o_dft[1]
        except:
            log_df.loc['chked', 4] = False
            o_feedback += o_dft[2]
    else:
        o_feedback += o_dft[0]
            
    # 5
    o_feedback += "05 - "
    if log_df.loc['chked', 5] != True:
        try:
            ndf_lst['05_1'], ndf_lst['05_2'] = S5_data()
            log_df.loc['chked', 5] = True
            o_feedback += o_dft[1]
        except:
            log_df.loc['chked', 5] = False      
            o_feedback += o_dft[2]
    else:
        o_feedback += o_dft[0]
    
    
    
    # 6
    o_feedback += "06 - "
    if log_df.loc['chked', 6] != True:
        try:
            #상해 선물에서 웹사이트가 안뜨는 현상 종종 발생 - 뭔가 바꿔야 할 
            ndf_lst['06_1'], ndf_lst['06_2'] = S6_data(Input_Date, max_try=15)      # 상해 선물, 기준일 데이터 없으면 그 전날 찾을 때까지 반복(최대 5일전까지 - max_try)
            
            log_df.loc['chked', 6] = True
            
            # 칼럼 이름 바꾸기 - 중복 방지
            ndf_lst['06_1'].columns = ['Delivery month_1', 'Pre settle_1', 'Open_1', 'High_1', 'Low_1', 'Close_1']
            ndf_lst['06_2'].columns = ['Delivery month_2', 'Pre settle_2', 'Open_2', 'High_2', 'Low_2', 'Close_2']
            
            o_feedback += o_dft[1]
        except:
            log_df.loc['chked', 6] = False 
            o_feedback += o_dft[2]
    else:
        o_feedback += o_dft[0]
    
    
    # 7
    o_feedback += "07 - "
    if log_df.loc['chked', 7] != True:
        try:
            ndf_lst['07'] = S7_data()
            
            log_df.loc['chked', 7] = True
            o_feedback += o_dft[1]
        except:
            log_df.loc['chked', 7] = False 
            o_feedback += o_dft[2]
    else:
        o_feedback += o_dft[0]
            
    
    # 8
    o_feedback += "08 - "
    if log_df.loc['chked', 8] != True:
        try:
            ndf_lst['08_1'], ndf_lst['08_2'], ndf_lst['08_3'], ndf_lst['08_4'], ndf_lst['08_5'], ndf_lst['08_6'] = S8_data()
            
            log_df.loc['chked', 8] = True
            o_feedback += o_dft[1]
        except:
            log_df.loc['chked', 8] = False 
            o_feedback += o_dft[2]
    else:
        o_feedback += o_dft[0]
            
    
    # 9
    o_feedback += "09 - "
    if log_df.loc['chked', 9] != True:
        try:
            ndf_lst['09_1'], ndf_lst['09_2'] = S9_data(Input_Date, max_try=10)      # 마이스틸(9-2), 기준일 데이터 없으면 그 전날 찾을 때까지 반복(최대 5일전까지 - max_try)
            
            log_df.loc['chked', 9] = True
            o_feedback += o_dft[1]
        except:
            log_df.loc['chked', 9] = False 
            o_feedback += o_dft[2]
    else:
        o_feedback += o_dft[0]
            
            
    # 10
    o_feedback += "10 - "
    if log_df.loc['chked', 10] != True:
        try:
            ndf_lst['10'] = S10_data()
            
            log_df.loc['chked', 10] = True
            o_feedback += o_dft[1]
        except:
            log_df.loc['chked', 10] = False
            o_feedback += o_dft[2]
    else:
        o_feedback += o_dft[0]
            
    
    
    browser.quit()
    
    
    
    
    # 3) df 합치기 : old + new = sum : new data에 우선순위를 둬서 수집 -------------------------------------------------------------------------------------------
    sdf_lst = {}
    
    # odf_lst['2']
    # ndf_lst['2']
    
    # t_o = odf_lst['01']
    # t_n = ndf_lst['01']
    
    
    
    # if t_o is t_n:    
    #     print("똑같음")
    # else:
    #     t_s = pd.concat([t_o, t_n])
        
    
    for k in odf_lst.keys():
        
        sdf_lst[k] = odf_lst[k]
        
        try:
            
            if ndf_lst[k] is odf_lst[k]:    
                print("Data_" + k + " : 값이 변경되지 않음")
                continue
            
            
            temp = pd.concat([ndf_lst[k], odf_lst[k]])
            # temp = pd.concat([ndf_lst['1'], odf_lst['1']])
            
            temp = temp.loc[~temp.index.duplicated(keep='first')]
            s_df = temp.sort_index(ascending=True)
            
            sdf_lst[k] = s_df    
            
        except:
            pass
        
    
    
    # 4) 파일에 새로 쓰기 -------------------------------------------------------------------------------------------
    
    output_file = input_file  #'D:/★프로세스 혁신★/22.09월/철강 MI 실행 추진/철강MI_데이터 수집/data_collected.xlsx'
    
    f_writer=pd.ExcelWriter(output_file, engine='openpyxl')
    
    
    for k in sdf_lst.keys():
        # write_file(f_writer, k, sdf_lst[k])
        
        name_index = int(k[:2]) - 1
        sdf_lst[k].to_excel(f_writer, index=True, sheet_name=site_lst[name_index] + "_" + str(k))
        
    f_writer.save()
    # f_writer.close()
    
    print("<< 데이터 업데이트 진행사항 >>  *기준일: " + Input_Date)
    print(o_feedback)
    
    
    # log_file 업데이트 - log_df ------------------------------------
    # log_df['Date'][0] = Input_Date
    # log_df[6][0] = False   # 각 모듈 테스트 목적 
    log_df.to_excel(log_file, index=True)
    
    
    
    
    
    # 5) 10개 df를 1개로 통합, index 기준 / 대상: sdf_lst - 10개 다 True일 경우에만 !! -------------------------------------------------------------------------------------------
    # 다 성공했을 때만 data_lake 파일 만들지는 좀 고민됨
    chk_lst = list(log_df.iloc[0,1:])
    
    if False not in chk_lst:    # 해당일 다 크롤링 성공했을 때 생성
        
        temp_lst = sdf_lst
    
        lake_sum = pd.DataFrame()
    
        for temp in temp_lst.keys():
            # con_sum = pd.concat([con_sum, sdf_lst[temp]])
            lake_sum = pd.merge(lake_sum, temp_lst[temp], left_index=True, right_index=True, how='outer')
         
    
        # # index 중복여부 확인
        # lake_sum.index.is_unique
        # lake_sum.index.nunique() == len(lake_sum)
    
        # lake_sum.info()
    
    
        # 빈데이터 채우기
        lake_sum_2 = lake_sum.replace(0, np.nan)   # '0'은 nan으로 바꿔 놓음
        lake_sum_3 = lake_sum_2.fillna(method='ffill')
        lake_sum_3.isnull().sum()
        
        
        # 환율 컨버팅 - 칼럼 만들기
        lake_sum_3.columns
        
        try:
            # 중국 위안
            lake_sum_3['SB01032_2'] = lake_sum_3['SB01032'] / lake_sum_3['CNY_2']
            
            lake_sum_3['Close_1_2'] = lake_sum_3['Close_1'] / lake_sum_3['CNY_2']
            lake_sum_3['Close_2_2'] = lake_sum_3['Close_1'] / lake_sum_3['CNY_2']
            
            lake_sum_3['HRC avg_2'] = lake_sum_3['HRC avg'] / lake_sum_3['CNY_2']
            
            
            lake_sum_3['SHA_HRC'] = lake_sum_3['SHA_HRC'] / lake_sum_3['CNY_2']
            
            
            # 인도 루피아 - 대상 : TS01046, TJ_HRC
            lake_sum_3['TS01046'] = lake_sum_3['TS01046'] * lake_sum_3['INR_2']
            lake_sum_3['TJ_HRC'] = lake_sum_3['TJ_HRC'] * lake_sum_3['INR_2']
            
            
        except:
            pass
        
        
        # 파일 쓰기 - Data_Lake
        Lake_file = '/content/drive/MyDrive/1. Data_Lake.xlsx'
        
        lake_sum_3.to_excel(Lake_file, index=True)
        
    else:
        print("!! 실행결과 중 False가 존재하여 data_lake 파일을 생성하지 않음")

[S2 - 해운지수]
               BDI     BCI     BPI     BSI
Date                                      
2022-10-17  1843.0  2186.0  2088.0  1678.0
2022-10-14  1838.0  2166.0  2081.0  1690.0
2022-10-13  1818.0  2094.0  2088.0  1696.0
2022-10-12  1873.0  2199.0  2147.0  1708.0
2022-10-11  1904.0  2246.0  2196.0  1714.0

[S3 - 원유가]
            Dubai  Brent    WTI
Date                           
2022-10-14  92.12  91.63  85.61
2022-10-17  89.69  91.62  85.46

[S4 - 환율]
0              USD     CNY    INR     CNY_2     INR_2
Date                                                 
2022-10-17  1428.9  198.79  17.35  7.187987  0.012142

[S5 - 스틸데일리]
[5-1 열연]
            P_SS275  P_SM355    P_GS  H_SS275  I_SS275  I_SS355
Date                                                           
2022-10-14   1200.0   1260.0  1150.0   1200.0   1100.0   1160.0
2022-10-07   1200.0   1260.0  1150.0   1200.0   1110.0   1170.0
2022-09-30   1250.0   1310.0  1250.0   1250.0   1150.0   1210.0
2022-09-23   1200.0   1260.0  1

In [10]:
# 아래는 Platts 테스트 목적으로 작성함 - 테스트 성공!!

# -*- coding: utf-8 -*-
# """
# Created on Tue Mar 22 13:15:23 2022

# @author: jyoon.jeong
# """

# from typing import Any
# from urllib.parse import urljoin
# import datetime
# from datetime import date, timedelta
# from pprint import pprint

# import requests
# import pandas as pd


# DATE_FORMAT = '%Y%m%d'
# NETLOC = 'https://api.platts.com'

# # PROXIES = {"http": "http://90.8.50.30:8080/", "https": "http://90.8.50.30:8080/"}
# APPKEY = "EyldIGwzyApJUvnBgLCK"
# FIELDS = 'close,unspecified'

# FILEPATH = "./market_data.ods"


# # def insert_date_column(symbols: dict[str, dict[str, list[dict[str, Any]]]], read_date: date) -> list[dict[str, Any]]:
# #     return [{'date': read_date.isoformat()} | symbol for symbol in symbols["Symbols"]["Symbol"]]

# # def market_data_v1(date_filter, symbols: str, filepath: str, 
# #                    fields: str = FIELDS, appkey: str = APPKEY, proxies: dict[str, str] = PROXIES):
  
# def market_data_v1(date_filter, symbols, # filepath, 
#                    fields = FIELDS, appkey = APPKEY, proxies = PROXIES):
#     headers = {
#         'accept': 'application/json',
#         'appkey': appkey,
#     }

#     filter = f'Symbol[]={symbols}^{date_filter}'
    
#     params = {
#         'Fields': fields,
#         'Filter': filter,
#         'Sort': 'symbol:asc,assessDate:asc',
#         'PageSize': 10000
#     }
    
#     response = requests.get(urljoin(NETLOC, '/marketdata/v3/symbolData'),
#                                         headers=headers, params=params)
    
#     # response = requests.get(urljoin(NETLOC, '/marketdata/v3/symbolData'),
#     #                         headers=headers, params=params) #, 
#     #                         #verify=False) #proxies=proxies, 
#     print(response.json())
#     return response.json()["Content"]["MarketData"]["SymbolData"]["Rows"]["Row"] #
    


# # 받아올 데이터 - Symbols
# SYMBOLS = "PLVHA00,SB01032,SB01063,SB01083,SB01084,SB01119,SB01125,SB01126,SB01142,SB01152,SB01180,SB01195,SB01233,SB01261,SBO1001,STBLB00,STCBZ02,STHAM00,STHGM00,STHRE00,STHRZ02,STHSA00,STPGM00,STRAM00,STRGM00,TS01011,TS01043,TS01046"


# input_date = "2022-10-11"  # 기준일자 

# dinput_date = datetime.datetime.strptime(input_date, '%Y-%m-%d')  # datetime 변경

# # input_date 기준 
# datelist = pd.date_range(dinput_date+datetime.timedelta(days=-1+1), dinput_date)  #최근 10일에 대해 확인

# # date를 처리 위한 string 변환
# strlist = list(map(str, datelist))  # date -> string 변환



# # (Start) Platts API 값을 받아오는 부분 -----------------------------------------------------------------------------------
# pd_temp_i = pd.DataFrame()


# for one_date in strlist:

#     s_date = str(one_date[:10]).split("-")
#     s_year, s_mon, s_day = (s_date[0], s_date[1].zfill(2), s_date[2].zfill(2))
    
#     ss_date = s_year + s_mon + s_day
    
#     # date_filter = "Date>=20220922^Date<=20220922"
#     date_filter = "Date>="+ss_date+"^Date<="+ss_date
    
#     result = market_data_v1(date_filter, SYMBOLS, FIELDS, APPKEY) #, PROXIES)FILEPATH, 

    
#     # for i in range(0, len(result)):
    
#     try:
#         rows = result[0]['Symbols']['Symbol']
#         # print(rows)
#         # print(str(i) + " - ", str(len(rows)))
        
#         pd_temp_j = pd.DataFrame()
        
        
#         for j in range(0, len(rows)):
            
#             row = rows[j]
#             df = pd.DataFrame(row, index= [0])   #"2022-10-4"])
#             # print(j, " : ", df)
            
#             if df.iloc[0, 0] == "PLVHA00":
#                 temp = df.iloc[[0], [0,3]]
#             else:
#                 temp = df.iloc[[0], [0,2]]
                
#             t_temp = pd.DataFrame(index=range(1,2), columns={temp.iloc[0,0]})
#             t_temp.iloc[0,0] = temp.iloc[0,1]
#             t_temp['Date'] = ss_date  #datelist[i]
            
#             t_temp = t_temp.astype({'Date':'datetime64'})        
#             t_temp.set_index('Date', inplace=True)
            
#             pd_temp_j = pd.concat([pd_temp_j, t_temp], axis = 1)
            
#         pd_temp_i = pd.concat([pd_temp_i, pd_temp_j], axis = 0)
        
#     except:
#         pass

# # (End) Platts API 값을 받아오는 부분 -----------------------------------------------------------------------------------            
        
# print(pd_temp_i)
# # pd_temp_i

# # pd_temp_i.info()
# # pd_temp_i["STHGM00"].unique()
# # sd1 = pd_temp_i.replace('#N/A', '0')


# # nsd1 = sd1.astype('float')     # 최종 결과물
# # print(nsd1)


# # 요 위까지 완료됨, 데이터 정리하는 건 계속 더 해야 함..---------------------------------------------------






